In [ ]:
# Dependencies
from config import gkey, owkey
import matplotlib.pyplot as plt
import json
import requests
import random
import pandas as pd
import numpy as np
from pprint import pprint
from citipy import citipy

In [ ]:
# Open Weather API preparation
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build partial query URL
query_url = f"{url}appid={owkey}&units={units}&q="

In [ ]:
#Test URL request with one city
city = "malaga"
print(query_url + city)
response = requests.get(query_url + city).json()
pprint(response)
max_temp = response['main']['temp_max']
humidity = response['main']['humidity']
cloudiness = response['clouds']['all']
wind_speed = response['wind']['speed']
city_name = response['name']

#print(f"The max temp for {city} is: {max_temp}")
#print(f"The humidity for {city} is: {humidity}")
#print(f"The cloudiness for {city} is: {cloudiness}")
#print(f"The wind speed for {city} is: {wind_speed}")
print(f"City {city} is: {city_name}")

In [ ]:
#Generate a list of random coordinates
#Limiting coordinates to those from the Iberian Peninsula (I'm Spanish:))
lat_lower_limit = 36.00 
lat_higher_limit = 43.78
lng_lower_limit = -8.87
lng_higher_limit = 3.31

#Creating data frame full of zeros to store coordinates and nearest city
number_of_cities = 10
cities = pd.DataFrame(0, index=np.arange(number_of_cities), columns=['Lat', 'Lng', 'City'])
cities.head()

In [ ]:
for index, row in cities.iterrows():
    
    print("\r\nDrill #" + str(index))
    
    lat = random.uniform(lat_lower_limit, lat_higher_limit)
    lng = random.uniform(lng_lower_limit, lng_higher_limit)
    city = citipy.nearest_city(lat, lng)

    cities.loc[[index],['Lat']] = lat
    cities.loc[[index],['Lng']] = lng
    cities.loc[[index],['City']] = city.city_name
    print("%s: %s, %s" % (city.city_name, lat, lng))

cities.head()

In [ ]:
#Get Weather information for each city in data frame
#Max Temperature, Humidity, Cloudiness, Wind Speed

#Creating new columns in data frame
cities["Max Temperature"] = ""
cities["Humidity"] = ""
cities["Cloudiness"] = ""
cities["Wind Speed"] = ""

#Requesting weather info and filling the data frame
for index, row in cities.iterrows():
    
    #print("\r\nDrill #" + str(index))
    
    #cities.loc[[index],['City'] returns a series instead of the value from the specific row I'm looking for
    city = cities.at[index,'City']
    print(city)
    
    #print(query_url + city)
    response = requests.get(query_url + city).json()
    
    try:

        #Extracting the desired fields from the response
        max_temp = response['main']['temp_max']
        humidity = response['main']['humidity']
        cloudiness = response['clouds']['all']
        wind_speed = response['wind']['speed']

        #print(f"The max temp for {city} is: {max_temp}")
        #print(f"The humidity for {city} is: {humidity}")
        #print(f"The cloudiness for {city} is: {cloudiness}")
        #print(f"The wind speed for {city} is: {wind_speed}")

        #Filling the data frame
        cities.loc[[index],['Max Temperature']] = max_temp
        cities.loc[[index],['Humidity']] = humidity
        cities.loc[[index],['Cloudiness']] = cloudiness
        cities.loc[[index],['Wind Speed']] = wind_speed

        #Re-writing the city name with what OpenWeather provides for better formatting
        city_name = response['name']
        #print(response['name'])
        cities.loc[[index],['City']] = city_name

    except (KeyError, IndexError):
        print("Error with city data. Skipping")
        continue
    

In [ ]:
cities.head()

In [ ]:
#Save the dataframe into a csv file
# Save the DataFrame as a csv
cities.to_csv("output_analysis/cities.csv",
                     encoding="utf-8", index=False)

In [ ]:
cities.dtypes

In [ ]:
#city latitude vs max temperature
#Convert Max Temperature column to float
cities["Max Temperature"] = pd.to_numeric(cities["Max Temperature"], errors = 'ignore')
# Build a scatter plot for each data type
plt.scatter(cities["Lat"],
            cities["Max Temperature"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Temperature")

# Incorporate the other graph properties
plt.title("City Latitude vs Max Temperature (°C)")
plt.ylabel("Max Temperature")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([36, 43.8])
plt.ylim([0, 25])

# Save the figure
plt.savefig("output_analysis/Latitude_vs_MaxTemperature.png")

# Show plot
plt.show()

In [ ]:
#city latitude vs humidity
#Convert Humidity column to float
cities["Humidity"] = pd.to_numeric(cities["Humidity"], errors = 'ignore')
# Build a scatter plot for each data type
plt.scatter(cities["Lat"],
            cities["Humidity"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Humidity")

# Incorporate the other graph properties
plt.title("City Latitude vs Humidity (%)")
plt.ylabel("Humidity")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([36, 43.8])
plt.ylim([40, 100])

# Save the figure
plt.savefig("output_analysis/Latitude_vs_Humidity.png")

# Show plot
plt.show()

In [ ]:
#city latitude vs Cloudiness
#Convert Humidity column to float
cities["Cloudiness"] = pd.to_numeric(cities["Cloudiness"], errors = 'ignore')
# Build a scatter plot for each data type
plt.scatter(cities["Lat"],
            cities["Cloudiness"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Cloudiness")

# Incorporate the other graph properties
plt.title("City Latitude vs Cloudiness (%)")
plt.ylabel("Cloudiness")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([36, 43.8])
plt.ylim([40, 100])

# Save the figure
plt.savefig("output_analysis/Latitude_vs_Cloudiness.png")

# Show plot
plt.show()

In [ ]:
#city latitude vs Wind Speed
#Convert Humidity column to float
cities["Wind Speed"] = pd.to_numeric(cities["Wind Speed"], errors = 'ignore')
# Build a scatter plot for each data type
plt.scatter(cities["Lat"],
            cities["Wind Speed"],
            edgecolor="black", linewidths=1, marker="o",
            alpha=0.8, label="Wind Speed")

# Incorporate the other graph properties
plt.title("City Latitude vs Wind Speed (m/s)")
plt.ylabel("Wind Speed")
plt.xlabel("Latitude")
plt.grid(True)
plt.xlim([36, 43.8])
plt.ylim([0, 15])

# Save the figure
plt.savefig("output_analysis/Latitude_vs_Wind Speed.png")

# Show plot
plt.show()